<a href="https://colab.research.google.com/github/Harshit-jain-1/LLMs-model/blob/main/code_generation_model_llms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
!pip install transformers dataset

In [54]:
!pip install transformers[torch] accelerate -U

In [55]:
!pip install PyGithub datasets

In [56]:
# import libraries
from github import Github
import re
from datasets import Dataset

In [57]:
# initialize PyGithub with the github token
g = Github("you token name")

In [58]:
# specify the repository
repo = g.get_repo("owner name /repository name")

# function to extract Python functions from a script
def extract_functions_from_code(code):
    pattern = re.compile(r"def\s+(\w+)\s*\(.*\):")
    functions = pattern.findall(code)
    return functions

# fetch Python files from the repository
python_files = []
contents = repo.get_contents("")
while contents:
    file_content = contents.pop(0)
    if file_content.type == "dir":
        contents.extend(repo.get_contents(file_content.path))
    elif file_content.path.endswith(".py"):
        python_files.append(file_content)

# extract functions and create dataset
data = {"code": [], "function_name": []}
for file in python_files:
    code = file.decoded_content.decode("utf-8")
    functions = extract_functions_from_code(code)
    for function in functions:
        data["code"].append(code)
        data["function_name"].append(function)

# create a Hugging Face dataset
dataset = Dataset.from_dict(data)

# save the dataset to disk
dataset.save_to_disk("generation_dataset")

print("Dataset created and saved to disk.")

Saving the dataset (0/1 shards): 0 examples [00:00, ? examples/s]

Dataset created and saved to disk.


In [59]:
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

In [60]:
# Function to load and inspect the dataset
def load_and_inspect_dataset(dataset_path):
    try:
        # Load the dataset from disk
        dataset = load_from_disk(dataset_path)
        print("Dataset loaded successfully.")
    except Exception as e:
        print(f"Error loading dataset: {e}")
        return None

    # Inspect dataset splits and contents
    print("Dataset keys:", dataset.keys())  # Check for 'train' and 'test' splits
    if 'train' in dataset:
        print("Training dataset size:", len(dataset['train']))
        print("First few examples from the training dataset:", dataset['train'][:5])  # Show a sample
    else:
        print("No 'train' split found in the dataset.")

    if 'test' in dataset:
        print("Testing dataset size:", len(dataset['test']))
        print("First few examples from the testing dataset:", dataset['test'][:5])  # Show a sample
    else:
        print("No 'test' split found in the dataset.")

    return dataset

# Function to preprocess the dataset
def preprocess_function(examples, tokenizer):
    # Tokenize the input code (adjust the max_length as needed)
    tokenized = tokenizer(examples['code'], truncation=True, padding='max_length', max_length=512)

    # Example: Add labels based on 'function_name' (customize this logic for your task)
    tokenized['label'] = [1 if fn == 'desired_function_name' else 0 for fn in examples['function_name']]

    return tokenized

# Function to train the model
def train_model(dataset, model_name):
    if dataset is None:
        print("Error: Dataset is empty or could not be loaded.")
        return

    # Split the dataset into train and test sets
    dataset = dataset.train_test_split(test_size=0.1)

    # Check dataset size after splitting
    print("Loaded training dataset size:", len(dataset['train']))
    print("Loaded testing dataset size:", len(dataset['test']))

    # Load the tokenizer for the model
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Apply the preprocessing function to the dataset
    tokenized_datasets = dataset.map(lambda examples: preprocess_function(examples, tokenizer), batched=True)

    # Check the size of the tokenized dataset
    print("Train dataset size after tokenization:", len(tokenized_datasets['train']))
    print("Test dataset size after tokenization:", len(tokenized_datasets['test']))

    # Check the columns of the tokenized dataset
    print("Tokenized dataset columns:", tokenized_datasets['train'].column_names)

    # Set the format for PyTorch
    tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    # Initialize the model
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir="./results",  # Output directory for results
        per_device_train_batch_size=8,  # Training batch size
        num_train_epochs=3,  # Number of epochs
        save_steps=10_000,  # Save checkpoints every 10,000 steps
        save_total_limit=2,  # Only save the last 2 checkpoints
        evaluation_strategy="epoch",  # Evaluate at the end of each epoch
        logging_dir="./logs",  # Directory for storing logs
        logging_steps=500,  # Log every 500 steps
        remove_unused_columns=False,  # Keep all columns in the dataset
    )

    # Initialize the Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets['train'],
        eval_dataset=tokenized_datasets['test'],
    )

    # Check if the dataset has any samples for training and testing
    if len(tokenized_datasets['train']) == 0:
        raise ValueError("Error: Training dataset is empty.")
    if len(tokenized_datasets['test']) == 0:
        raise ValueError("Error: Testing dataset is empty.")

    # Start training the model
    trainer.train()

# Main execution flow
dataset_path = "generation_dataset"

In [61]:
# define a function to generate code using the fine-tuned model
def generate_code(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs['input_ids'], max_length=max_length)
    generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_code

# test the model with a code generation prompt
prompt = "def merge_sort(arr):"
generated_code = generate_code(prompt)

print("Generated Code:")
print(generated_code)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Code:
def merge_sort(arr):
    if len(arr) <= 1:
        return arr
    mid = len(arr) // 2
    left = merge_sort(arr[:mid])
    right = merge_sort(arr[mid:])
    return merge(left, right)

def merge(left, right):
    result = []
    while left and right:
        if left[0] < right[0]:

